## Load Packages

In [1]:

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Utility Function

Recall: the CRRA utility function is $U(W) = \frac{W^{1-k}}{1-k}$

In [3]:
U(x,k) =  x.^(1-k)/(1-k)    #CRRA utility function, k is the risk aversion

U (generic function with 1 method)

In [4]:
W = linspace(0.8,1.2,25)


plot(W,U(W,2),color=:red,linewidth=2,legend=false)
title!("Utility function, CRRA")
xlabel!("wealth")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,368.504 62.4016,346.425 85.4331,325.228 108.465,304.863 131.496,285.282 154.528,266.439 177.559,248.294 200.591,230.809 223.622,213.948 246.654,197.679 
 269.685,181.971 292.717,166.796 315.748,152.126 338.78,137.937 361.811,124.206 384.843,110.911 407.874,98.0315 430.906,85.5482 453.937,73.4431 476.969,61.6994 
 500,50.3011 523.031,39.2331 546.063,28.4814 569.094,18.0326 592.126,7.87402 
 "/>

# Expected Utility

Recall: if $\pi_s$ is the probability of outcome $W_s$ and there are $S$ possible outcomes, then

$\text{E}W = \sum\nolimits_{s=1}^{S} \pi_{s}W_{s}$ and 

$\text{E}U(W) = \sum\nolimits_{s=1}^{S} \pi_{s}U(W_{s})$

Concavity of the utility function means that 
$\text{E} U(W) < U(\text{E} W)$

In [5]:
EU(x,π,k) = sum(π .* U(x,k))               #expected utility

EU (generic function with 1 method)

In [6]:
W₁ = 0.85               #possible outcomes
W₂ = 1.15
π₁ = 0.5                #probabilities of outcomes 
π₂ = 1 - π₁

xx = [W₁ U(W₁,2);
      W₂ U(W₂,2)]
println("Possible wealth levels and the corresponding utility")
printmat(xx)

UtilExpW = U(sum([π₁ π₂].*[W₁ W₂]),2)
ExpUtil  = EU([W₁ W₂],[π₁ π₂],2)
printlnPs("Utility at expected value and Expected utility: ",[UtilExpW ExpUtil])

Possible wealth levels and the corresponding utility
     0.850    -1.176
     1.150    -0.870

Utility at expected value and Expected utility:     -1.000    -1.023


In [7]:
plot(W,U(W,2),color=:red,linewidth=2,label="Utility fn")
scatter!([W₁;W₂],U([W₁;W₂],2),color=:green,label="possible outcomes")
scatter!([1.0],[ExpUtil],color=:blue,marker=:rect,label="Expected utility")
plot!(ylims=(-1.3,-0.7))
title!("Utility function, CRRA")
xlabel!("wealth")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 1.2 
 
 
 -1.3 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 -0.8 
 
 
 -0.7 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 55.0141,338.451 76.7419,323.119 98.4698,308.399 120.198,294.257 141.925,280.658 163.653,267.573 185.381,254.972 207.109,242.83 228.837,231.121 250.565,219.823 
 272.292,208.915 294.02,198.376 315.748,188.189 337.476,178.336 359.204,168.8 380.932,159.568 402.659,150.623 424.387,141.954 446.115,133.548 467.843,125.393 
 489.571,117.477 511.298,109.791 533.026,102.325 554.754,95.0686 576.482,88.014 
 "/>
 
 
 
 
 
 
 
 
 
 
 Utility fn 
 
 
 
 
 possible outcomes 
 
 
 
 
 Expected utility

# Certainty Equivalent

Recall: the certainty equivalent (here denoted $P$) is the sure (non-stochastic) value that solves 

$U(P) = \text{E}U(W)$

The code below solves for $P$ by a numerial search for different definitions of the $U$ function (different risk aversions).

Notice that the expected wealth level is 1.

In [8]:
using Roots

println("risk aversion and certainly equivalent")
kM = [0.0001;2;5;10;25]
for k in kM
    ceq = fzero(P->EU([W₁ W₂],[π₁ π₂],k) - P^(1-k)/(1-k),[0.5,1.5])
    printlnPs([k ceq])
end

risk aversion and certainly equivalent
     0.000     1.000
     2.000     0.978
     5.000     0.947
    10.000     0.912
    25.000     0.875


# Portfolio Choice with One Risky Asset

Recall: the investor solves $\max\nolimits_{v}\text{E}\ln R_{p}\text{, with }R_{p}=vR_{1} + (1-v)R_{f}$

In the example below, there are two possible outcomes for $R_{1}$ (-0.2 and 0.5) with equal probabilities

In [9]:
function EUc(v,π,Re,Rf)
    R = Re + Rf
    Rp = v*R + (1-v)*Rf             #portfolio return
    EU = sum(π .* log.(1+Rp))        #expected utility
    return EU
end

EUc (generic function with 1 method)

In [10]:
v = linspace(-1,1,101)
L = length(v)
EUv = fill(NaN,L)
for i = 1:L
    EUv[i] = EUc(v[i],[0.5 0.5],[-0.3 0.4],0.1)
end    
    
plot(v,EUv,color=:red,linewidth=2,legend=false)
title!("Expected Utility")
xlabel!("weight on risky asset")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 Expected Utility 
 
 
 weight on risky asset 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,368.504 44.8976,357.489 50.4252,346.703 55.9528,336.14 61.4803,325.796 67.0079,315.668 72.5354,305.753 78.063,296.045 83.5906,286.543 89.1181,277.242 
 94.6457,268.14 100.173,259.232 105.701,250.517 111.228,241.991 116.756,233.651 122.283,225.495 127.811,217.52 133.339,209.724 138.866,202.103 144.394,194.656 
 149.921,187.381 155.449,180.274 160.976,173.334 166.504,166.559 172.031,159.947 177.559,153.496 183.087,147.204 188.614,141.07 194.142,135.09 199.669,129.265 
 205.197,123.591 210.724,118.068 216.252,112.694 221.78,107.468 227.307,102.388 232.835,97.4522 238.362,92.6603 243.89,88.0105 249.417,83.5016 254.945,79.1326 
 260.472,74.9021 266,70.8092 271.528,66.8528 277.055,63.0319 282.583,59.3455 288.11,55.7927 293.638,52.3725 299.165,49.0842 304.693,45.9269 310.22,42.8999 
 315.748,40.0023 321.276,37.2335 326.803,34.5928 332.331,32.0796 337.858,29.6932 343.386,27.4331 348.913,25.2987 354.441,23.2896 359.969,21.4052 365.496,19.6451 
 371.024,18.0088 376.551,16.4961 382.079,15.1064 387.606,13.8396 393.134,12.6953 398.661,11.6732 404.189,10.7731 409.717,9.99474 415.244,9.33802 420.772,8.80274 
 426.299,8.38878 431.827,8.09605 437.354,7.92448 442.882,7.87402 448.409,7.94466 453.937,8.13643 459.465,8.44936 464.992,8.88352 470.52,9.43903 476.047,10.116 
 481.575,10.9146 487.102,11.835 492.63,12.8775 498.157,14.0422 503.685,15.3295 509.213,16.7396 514.74,18.2729 520.268,19.9298 525.795,21.7106 531.323,23.6158 
 536.85,25.6457 542.378,27.801 547.906,30.0821 553.433,32.4896 558.961,35.0241 564.488,37.686 570.016,40.4763 575.543,43.3954 581.071,46.4441 586.598,49.6231 
 592.126,52.9334 
 "/>

Notice that this problem can be solved by pen and paper, but it 
becomes very difficult when the number of states increase --- and even worse when there are many assets. We therefore apply a numerical optimization algorithm.

In [11]:
using Optim          #minimization routines

#minimize -EU()
Sol = optimize(v-> -EUc(v,[0.5 0.5],[-0.3 0.4],0.1),-1,1)   #try v values between -1 and 1
printlnPs("argmin -EUc(): ",Optim.minimizer(Sol))

argmin -EUc():      0.458


# Portfolio Choice with Several Risky Assets

This optimization problem has two risky assets and a general CRRA utility function. Numerical optimization is still easy.

In [12]:
function EUe(v,π,R,Rf,γ)
    Rp = v'R + (1-sum(v))*Rf            #portfolio return
    EU = sum(π .* (1+Rp).^(1-γ)/(1-γ))  #expected utility
    EUNeg = -EU                         #since optimize() minimizes
    return EUNeg
end

EUe (generic function with 1 method)

In [13]:
ROutComes = [-0.03 0.08 0.20;           #cell[i,j] is asset i in state j
             -0.04 0.22 0.15]
Sol = optimize(v->EUe(v,[1/3 1/3 1/3],ROutComes,0.065,5),[-0.6;1.2])         #try v values between -1 and 1
printlnPs("optimal weights from max EUe(): ",Optim.minimizer(Sol))

optimal weights from max EUe():     -0.725     1.316


# From Chapter on Mean-Variance Analysis

In [14]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF
    n   = length(μ)
    μe  = μ - Rf              #excess returns            
    Σ_1 = inv(Σ)
    w   = (mustar-Rf)/(μe'Σ_1*μe)[1] .* Σ_1*μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

# Mean-Variance and the Telser Criterion

Let $\mu$ be a vector of expected returns and $\Sigma$ be the covariance matrix of the investible asssets.

The Telser criterion solves the problem

$\max_{v} \mu_{p} \> \text{ subject to} \>  \text{VaR}_{95\%} < 0.1$,

where $\mu_{p} = v'\mu+(1-v)R_f$ is the expected portfolio return.

If the returns are normally distributed then 

$\text{VaR}_{95\%} = -(\mu_p - 1.64\sigma_p)$,

where $\sigma_p = \sqrt{v'\Sigma v}$ is the standard deviation of the portfolio return.

The figure below illustrates that the optimal portfolio is on the CLM (when the returns are normally distributed)

In [15]:
μ = [9; 6]/100                     #means
Σ = [ 256  0;
      0    144]/10000
Rf = 1/100

mustar  = linspace(Rf,0.1,101)
L       = length(mustar)
StdRp   = fill(NaN,L)
StdRpRf = fill(NaN,L)
for i = 1:L
    StdRp[i]   = MVCalc(mustar[i],μ,Σ)[1]
    StdRpRf[i] = MVCalcRf(mustar[i],μ,Σ,Rf)[1]
end   

VaRRestr = -0.1 + 1.64*StdRpRf

println("let's plot it")

let's plot it


In [16]:
plot(StdRpRf*100,mustar*100,color=:blue,linewidth=2,label="CML",xlims=(0,15),ylims=(0,15))
plot!(StdRp*100,mustar*100,color=:red,linewidth=2,label="MVF")
plot!(StdRpRf*100,VaRRestr*100,line=(:dot,2),color=:green,label="VaR restriction")
title!("Mean vs standard deviation (with Rf)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation (with Rf) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,344.462 44.4657,342.298 49.5614,340.134 54.6571,337.971 59.7527,335.807 64.8484,333.643 69.9441,331.479 75.0397,329.315 80.1354,327.152 85.2311,324.988 
 90.3267,322.824 95.4224,320.66 100.518,318.497 105.614,316.333 110.709,314.169 115.805,312.005 120.901,309.841 125.996,307.678 131.092,305.514 136.188,303.35 
 141.283,301.186 146.379,299.023 151.475,296.859 156.57,294.695 161.666,292.531 166.762,290.367 171.857,288.204 176.953,286.04 182.049,283.876 187.144,281.712 
 192.24,279.549 197.336,277.385 202.431,275.221 207.527,273.057 212.623,270.893 217.718,268.73 222.814,266.566 227.91,264.402 233.005,262.238 238.101,260.075 
 243.197,257.911 248.292,255.747 253.388,253.583 258.484,251.419 263.579,249.256 268.675,247.092 273.771,244.928 278.866,242.764 283.962,240.601 289.058,238.437 
 294.153,236.273 299.249,234.109 304.345,231.945 309.44,229.782 314.536,227.618 319.632,225.454 324.727,223.29 329.823,221.127 334.919,218.963 340.014,216.799 
 345.11,214.635 350.206,212.471 355.301,210.308 360.397,208.144 365.493,205.98 370.588,203.816 375.684,201.652 380.78,199.489 385.875,197.325 390.971,195.161 
 396.067,192.997 401.162,190.834 406.258,188.67 411.354,186.506 416.449,184.342 421.545,182.178 426.641,180.015 431.736,177.851 436.832,175.687 441.928,173.523 
 447.023,171.36 452.119,169.196 457.215,167.032 462.31,164.868 467.406,162.704 472.502,160.541 477.597,158.377 482.693,156.213 487.789,154.049 492.884,151.886 
 497.98,149.722 503.076,147.558 508.171,145.394 513.267,143.23 518.363,141.067 523.458,138.903 528.554,136.739 533.65,134.575 538.745,132.412 543.841,130.248 
 548.937,128.084 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 1574.36,344.462 1552.85,342.298 1531.37,340.134 1509.9,337.971 1488.44,335.807 1467.01,333.643 1445.6,331.479 1424.22,329.315 1402.85,327.152 1381.51,324.988 
 1360.19,322.824 1338.9,320.66 1317.64,318.497 1296.41,316.333 1275.21,314.169 1254.04,312.005 1232.91,309.841 1211.81,307.678 1190.75,305.514 1169.73,303.35 
 1148.75,301.186 1127.82,299.023 1106.93,296.859 1086.1,294.695 1065.32,292.531 1044.59,290.367 1023.92,288.204 1003.32,286.04 982.791,283.876 962.332,281.712 
 941.95,279.549 921.651,277.385 901.442,275.221 881.328,273.057 861.317,270.893 841.415,268.73 821.633,266.566 801.978,264.402 782.462,262.238 763.094,260.075 
 743.889,257.911 724.858,255.747 706.017,253.583 687.383,251.419 668.974,249.256 650.81,247.092 632.914,244.928 615.311,242.764 598.029,240.601 581.097,238.437 
 564.551,236.273 548.427,234.109 532.768,231.945 517.618,229.782 503.028,227.618 489.052,225.454 475.75,223.29 463.183,221.127 451.421,218.963 440.534,216.799 
 430.594,214.635 421.676,212.471 413.853,210.308 407.194,208.144 401.764,205.98 397.618,203.816 394.802,201.652 393.347,199.489 393.27,197.325 394.573,195.161 
 397.239,192.997 401.239,190.834 406.529,188.67 413.054,186.506 420.751,184.342 429.551,182.178 439.381,180.015 450.167,177.851 461.836,175.687 474.316,173.523 
 487.54,171.36 501.444,169.196 515.969,167.032 531.059,164.868 546.664,162.704 562.738,160.541 579.239,158.377 596.13,156.213 613.375,154.049 630.944,151.886 
 648.808,149.722 666.943,147.558 685.326,145.394 703.936,143.23 722.755,141.067 741.765,138.903 760.952,136.739 780.302,134.575 799.803,132.412 819.443,130.248 
 839.211,128.084 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 39.3701,608.924 44.4657,603.472 49.5